<a href="https://colab.research.google.com/github/UmarAzamQureshi/World-Bank-Procurement-Monitor/blob/main/World_Bank_Procurement_Monitor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
# Install required packages
!apt-get update > /dev/null
!apt install chromium-chromedriver > /dev/null
!pip install pandas selenium beautifulsoup4 openpyxl streamlit> /dev/null
!npm install -g localtunnel
!pip install streamlit pyngrok --quiet
!pip install xlsxwriter


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


⠙⠹⠸⠼⠴
changed 22 packages in 653ms
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴Requirement already satisfied: xlsxwriter in /usr/local/lib/python3.11/dist-packages (3.2.5)


In [45]:
%%writefile app.py
import streamlit as st
import pandas as pd
import re
import time
import logging
from io import BytesIO
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ----------------------
# Selenium Setup
# ----------------------
def get_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--window-size=1920,1080")
    return webdriver.Chrome(options=chrome_options)

# ----------------------
# Scrape World Bank Data
# ----------------------
@st.cache_data(show_spinner=True)
def scrape_procurement_notices():
    base_url = "https://projects.worldbank.org"
    start_url = "https://projects.worldbank.org/en/projects-operations/procurement"

    driver = get_driver()
    driver.get(start_url)
    time.sleep(5)

    data = []
    page = 1

    while True:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        rows = soup.select("table tbody tr")

        if not rows:
            break

        for row in rows:
            cols = row.find_all("td")
            if len(cols) < 6:
                continue
            detail_link = cols[0].find("a")
            detail_url = base_url + detail_link["href"] if detail_link else ""
            data.append({
                "description": cols[0].get_text(strip=True),
                "country": cols[1].get_text(strip=True),
                "project_title": cols[2].get_text(strip=True),
                "notice_type": cols[3].get_text(strip=True),
                "language": cols[4].get_text(strip=True),
                "published_date": cols[5].get_text(strip=True),
                "detail_url": detail_url
            })

        try:
            next_button = driver.find_element(By.LINK_TEXT, "Next")
            if "disabled" in next_button.get_attribute("class"):
                break
            next_button.click()
            page += 1
            time.sleep(3)
        except:
            break

    driver.quit()
    return pd.DataFrame(data)

# ----------------------
# Scoring Function
# ----------------------
def score_projects(df: pd.DataFrame) -> pd.DataFrame:
    # Define domain-specific keywords with optional weights
    keywords = {
        "road": 10,
        "highway": 15,
        "bridge": 10,
        "traffic": 10,
        "transport planning": 20,
        "transport": 10,
        "environmental impact": 15,
        "social impact": 10,
        "EIA": 15,
        "ESIA": 10
    }

    def compute_score(text):
        if pd.isna(text):
            return 0
        text = text.lower()
        score = 0
        for keyword, weight in keywords.items():
            if keyword in text:
                score += weight
        return min(score, 100)  # Cap the score at 100

    df["score"] = df["description"].apply(compute_score)
    return df


# ----------------------
# Convert to Excel Bytes
# ----------------------
def to_excel_bytes(df, save_path=None):
    output = BytesIO()
    with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
        df.to_excel(writer, index=False, sheet_name="WorldBankData")
    output.seek(0)
    if save_path:
        with open(save_path, "wb") as f:
            f.write(output.read())
        output.seek(0)  # Rewind for download
    return output.read()

# ----------------------
# Streamlit UI
# ----------------------
st.set_page_config(page_title="World Bank Procurement Monitor", layout="wide", initial_sidebar_state="auto")

# ----------------------
# Custom CSS Styling
# ----------------------
custom_css = """
    <style>
    .header-title {
        display: flex;
        justify-content: space-between;
        align-items: center;
        padding-bottom: 1rem;
        border-bottom: 1px solid #444;
    }
    .header-title h1 {
        margin: 0;
        font-size: 2rem;
    }
    .filter-section {
        text-align: right;
        padding: 1rem;
    }
    .footer {
        margin-top: 2rem;
        padding-top: 1rem;
        border-top: 1px solid #444;
        text-align: center;
        font-size: 0.9rem;
        color: gray;
    }
    </style>
"""

st.markdown(custom_css, unsafe_allow_html=True)

# ----------------------
# Dark Mode Toggle
# ----------------------
dark_mode = st.toggle("Dark Mode 🌙", value=False, key="dark_mode")

if dark_mode:
    st.markdown(
        """
        <style>
        body {
            background-color: #111;
            color: white;
        }
        .stApp {
            background-color: #111;
        }
        </style>
        """,
        unsafe_allow_html=True,
    )

# ----------------------
# Header Title
# ----------------------
st.markdown("""
<div class="header-title">
    <h1>World Bank Procurement Monitor</h1>
</div>
""", unsafe_allow_html=True)

# ----------------------
# Step 1: Scrape and Download
# ----------------------
st.subheader("Step 1: Scrape and Download Opportunities")
if st.button("Scrape World Bank Notices"):
    scraped_df = scrape_procurement_notices()
    if not scraped_df.empty:
        scored_df = score_projects(scraped_df)
        st.success(f"Scraped {len(scored_df)} notices.")
        save_path = f"/content/world_bank_opportunities_{datetime.today().date()}.xlsx"
        excel_data = to_excel_bytes(scored_df, save_path=save_path)
        st.info(f"Excel file saved to: {save_path}")
        st.download_button(
            label="Download Excel",
            data=excel_data,
            file_name=f"world_bank_procurement_{datetime.today().date()}.xlsx",
            mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        )
    else:
        st.warning("No data scraped.")

# ----------------------
# Step 2: Upload and Filter
# ----------------------
st.markdown("---")
st.subheader("Step 2: Upload Scraped Excel File")
uploaded_file = st.file_uploader("Upload Excel File", type=["xlsx"])
if uploaded_file:
    df = pd.read_excel(uploaded_file)

    st.subheader("Step 3: Filter Opportunities")

    col1, col2 = st.columns([2, 1])
    with col2:
        st.markdown("<div class='filter-section'>", unsafe_allow_html=True)
        country_filter = st.multiselect("Filter by Country", sorted(df["country"].dropna().unique()))
        score_filter = st.slider("Filter by Score", min_value=0, max_value=100, value=(0, 100))
        keyword_search = st.text_input("Search in Description")
        st.markdown("</div>", unsafe_allow_html=True)

    filtered_df = df.copy()
    if country_filter:
        filtered_df = filtered_df[filtered_df["country"].isin(country_filter)]
    if score_filter:
        filtered_df = filtered_df[(filtered_df["score"] >= score_filter[0]) & (filtered_df["score"] <= score_filter[1])]
    if keyword_search:
        filtered_df = filtered_df[filtered_df["description"].str.contains(keyword_search, case=False, na=False)]


    with col1:
        st.markdown(f"#### Showing {len(filtered_df)} result(s)")
        st.dataframe(filtered_df, use_container_width=True)

    download_excel = to_excel_bytes(filtered_df)
    st.download_button(
        label="Download Filtered Data",
        data=download_excel,
        file_name=f"filtered_procurement_{datetime.today().date()}.xlsx",
        mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    )
else:
    st.info("Upload an Excel file after scraping to continue filtering.")

# ----------------------
# Footer
# ----------------------
st.markdown("""
<div class="footer">
    Developed by Umar Azam Qureshi
</div>
""", unsafe_allow_html=True)

Overwriting app.py


In [47]:
# ----------------------
# Run Streamlit app in Colab using ngrok
# ----------------------

if 'google.colab' in str(get_ipython()):
    import threading
    import os
    from pyngrok import ngrok

    ngrok.kill()
    ngrok.set_auth_token("30pBkGhCermU5hrQ528wL4DTv17_39XyQy6n1MWLcjkEMi5qK")  # Replace with your actual token

    def run_app():
        os.system("streamlit run app.py --server.port 8501")

    threading.Thread(target=run_app).start()
    public_url = ngrok.connect(8501)
    st.markdown(f"\U0001F310 [Open Streamlit App]({public_url})")
    print("\U0001F310 Streamlit app is live at:", public_url)


2025-08-04 14:21:27.772 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-04 14:21:27.773 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-04 14:21:27.773 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


🌐 Streamlit app is live at: NgrokTunnel: "https://38fab0d4d8a3.ngrok-free.app" -> "http://localhost:8501"
